In [16]:
import json

basic = "/home/thiziri/Documents/DOCTORAT/SOFT/MatchZoo_latest/MatchZoo/examples/letor/config/accnnm_basic_config.config"
weights = "/projets/iris/PROJETS/WEIR/code/3rdYear/MatchZoo_latest/MatchZoo/examples/Letor/weights/concatenated/merge_levels"
data = "/projets/iris/PROJETS/WEIR/data/passages_ranking_indri/LETOR/concatenated/fold1"
predictions = "/projets/iris/PROJETS/WEIR/code/3rdYear/MatchZoo_latest/MatchZoo/examples/Letor/predictions/concatenated/merge_levels"

output = "/home/thiziri/Documents/DOCTORAT/OSIRIM/projets/iris/PROJETS/WEIR/code/3rdYear/MatchZoo_latest/MatchZoo/examples/Letor/config/concatenated/tested_configs_with_mergeLevels_fold1"

In [17]:
basic_config_s = open(basic, 'r').read()
basic_config_s = basic_config_s.replace("<weight_path>", weights).replace("<data_path>", data).replace("<predict_path>", 
                                                                                                       predictions)
basic_config = json.loads(basic_config_s)
basic_config["global"]

{'display_interval': 10,
 'learning_rate': 0.1,
 'model_type': 'PY',
 'num_iters': 600,
 'optimizer': 'adam',
 'save_weights_iters': 50,
 'test_weights_iters': 600,
 'weights_file': '/projets/iris/PROJETS/WEIR/code/3rdYear/MatchZoo_latest/MatchZoo/examples/Letor/weights/concatenated/merge_levels/accnnm_letor_ranking<model>.weights'}

In [18]:
context_num = [5, 10, 15, 20]
context_len = [basic_config["inputs"]["share"]["context_len"]]
context_embed = [50, 100, 200]
conv_dropout_rate = [0.0, 0.2]
lstm_dropout_rate = [0.0, 0.2]
pool_size = [1, 2, 3]
text1_attention = [True, False]
text2_attention = [True, False]
per_context = [True, False]
context_attention = [True, False]
merge_levels = [True]
parameters = [context_num, context_len, context_embed, conv_dropout_rate, lstm_dropout_rate, pool_size, text1_attention, 
              text2_attention, per_context, context_attention, merge_levels]
print("To variate: %d" % len(parameters))

To variate: 11


In [19]:
import itertools

combinations = [c for c in itertools.product(*parameters)]
print("%d possible configurations.\nExample: %s" % (len(combinations), str(combinations[0])))

to_eliminate = [c for c in combinations if c[7] is False and c[8]]
print("%d invalid combinations. " % len(to_eliminate))

final_combinations = [c for c in combinations if c not in to_eliminate]
print("%d valid combinations. " % len(final_combinations))

2304 possible configurations.
Example: (5, 5, 50, 0.0, 0.0, 1, True, True, True, True, True)
576 invalid combinations. 
1728 valid combinations. 


In [20]:
from tqdm import tqdm
import os

for c in tqdm(final_combinations):
    config = basic_config.copy()
    context_num, context_len, context_embed, conv_dropout_rate, lstm_dropout_rate, pool_size, if_attQ, if_attD, \
    if_per_context, if_context_attention, if_merge_levels = c
    attQ = "_attQ" if if_attQ else ''
    attD = "_attD" if if_attD else ''
    granul = "_perContext" if if_per_context else "_perWord" if if_attD else ''
    attContext = "_attP" if if_context_attention else ''
    merge_wp = "_mergeLevels" if if_merge_levels else ''
    model = "_p"+str(context_len)+"_ctxtNum"+str(context_num)+"_ctxtEmb"+str(context_embed)+"_convDrop" + \
            str(conv_dropout_rate)+"_lstmDrop"+str(lstm_dropout_rate)+"_pool"+str(pool_size)+attQ+attD+granul+\
            attContext + merge_wp
    # print(model)
    config["global"]["weights_file"] = config["global"]["weights_file"].replace("<model>", model)
    config["outputs"]["predict"]["save_path"] = config["outputs"]["predict"]["save_path"].replace("<model>", model)
    config["inputs"]["share"]["text2_maxlen"] = context_num * context_len
    config["inputs"]["share"]["context_num"] = context_num
    config["model"]["setting"]["context_embed"] = context_embed
    config["model"]["setting"]["conv_dropout_rate"] = conv_dropout_rate
    config["model"]["setting"]["lstm_dropout_rate"] = lstm_dropout_rate
    config["model"]["setting"]["pool_size"] = pool_size
    config["model"]["setting"]["text1_attention"] = if_attQ
    config["model"]["setting"]["text2_attention"] = if_attD
    config["model"]["setting"]["per_context"] = if_per_context
    config["model"]["setting"]["context_attention"] = if_context_attention
    config["model"]["setting"]["merge_levels"] = if_merge_levels
    with open(os.path.join(output, 'accnnm'+model+'.config'), 'w') as out:
        out.write(json.dumps(config, indent=2))

  0%|          | 0/1728 [00:00<?, ?it/s]

  0%|          | 1/1728 [00:00<03:12,  8.96it/s]

  0%|          | 5/1728 [00:00<01:19, 21.61it/s]

  0%|          | 8/1728 [00:00<01:13, 23.53it/s]

  1%|          | 11/1728 [00:00<01:09, 24.85it/s]

...

 99%|█████████▉| 1714/1728 [01:13<00:00, 23.33it/s]

 99%|█████████▉| 1717/1728 [01:13<00:00, 23.33it/s]

100%|█████████▉| 1721/1728 [01:13<00:00, 23.35it/s]

100%|█████████▉| 1724/1728 [01:13<00:00, 23.35it/s]

100%|██████████| 1728/1728 [01:13<00:00, 23.36it/s]

In [71]:
# create scripts OSIRIM

scripts = "/home/thiziri/Documents/DOCTORAT/OSIRIM/projets/iris/PROJETS/WEIR/code/3rdYear/MatchZoo_latest/scripts/Letor/train_predict/train/concatenated/tested_configs_fold1/in_gpu"

header_gpu = """#!/bin/sh
#SBATCH --job-name=train_accnnm<num>_f1_gpu               # Nom du Job
#SBATCH --ntasks=2                       # Nombre de Tasks : 1
#SBATCH --cpus-per-task=4                # Allocation de 6 CPU par Task
#SBATCH --output=train_accnnm<num>_f1_gpu.out
#SBATCH --partition=GPUNodes
#SBATCH --gres=gpu:4
#SBATCH --mem-per-cpu=8000M

#SBATCH --mail-type=ALL                  # Notification par mail des évènements concernant le job : début d’exécution, fin,…
#SBATCH --mail-user=thiziri.belkacem@irit.fr

# /projets/iris/PROJETS/WEIR/code/3rdYear/MatchZoo_latest/scripts/Letor/train_predict/train/concatenated/tested_configs_fold1

cd /projets/iris/PROJETS/WEIR/code/3rdYear/MatchZoo_latest/MatchZoo/
"""
cmd = "srun -n1 -N1 keras-py3-tf /projets/iris/PROJETS/WEIR/python_env/inherited/gpu_supported/python3/bin/python3 matchzoo/main_gpu.py --gpus 2 --phase train --model_file "
num = 0
new_script = True
limit = 12
for file in tqdm(os.listdir(output)):
    num += 1
    if new_script:
        out = open(os.path.join(scripts, "run_accnnm_"+str(num)+".sh"), 'w')
        out.write(header_gpu.replace("<num>", str(num)))
        new_script = False
    line = cmd + os.path.join(output, file).replace("/home/thiziri/Documents/DOCTORAT/OSIRIM", '') + \
           " > /projets/iris/PROJETS/WEIR/code/3rdYear/MatchZoo_latest/MatchZoo/examples/Letor/train_logs/concatenated/tested_configs_fold1/" + file.replace(".config", ".log") + '&'
    out.write(line + '\n')
    if num % 2 == 0:
        out.write('wait\n')
    if num % limit == 0 or len(os.listdir(output)) - num < limit:
        new_script = True
        # if len(os.listdir(output)) - num < limit:
out.close()

  0%|          | 0/2304 [00:00<?, ?it/s]

  1%|          | 25/2304 [00:00<00:09, 246.53it/s]

  3%|▎         | 59/2304 [00:00<00:07, 292.92it/s]

  4%|▎         | 85/2304 [00:00<00:08, 277.02it/s]

  5%|▍         | 109/2304 [00:00<00:08, 266.57it/s]

...

 95%|█████████▍| 2185/2304 [00:08<00:00, 266.32it/s]

 96%|█████████▋| 2221/2304 [00:08<00:00, 267.04it/s]

 98%|█████████▊| 2250/2304 [00:08<00:00, 262.20it/s]

 99%|█████████▉| 2281/2304 [00:08<00:00, 262.19it/s]

100%|██████████| 2304/2304 [00:09<00:00, 255.22it/s]

In [72]:
# create scripts OSIRIM

scripts = "/home/thiziri/Documents/DOCTORAT/OSIRIM/projets/iris/PROJETS/WEIR/code/3rdYear/MatchZoo_latest/scripts/Letor/train_predict/train/concatenated/tested_configs_fold1/in_cpu"

header = """#!/bin/sh
#SBATCH --job-name=train_accnnm<num>_f1_cpu               # Nom du Job
#SBATCH --ntasks=2                       # Nombre de Tasks : 1
#SBATCH --cpus-per-task=12                # Allocation de 6 CPU par Task
#SBATCH --output=train_accnnm<num>_f1_cpu.out
#SBATCH --partition=24CPUNodes
#SBATCH --mem-per-cpu=7500M

#SBATCH --mail-type=ALL                  # Notification par mail des évènements concernant le job : début d’exécution, fin,…
#SBATCH --mail-user=thiziri.belkacem@irit.fr

# /projets/iris/PROJETS/WEIR/code/3rdYear/MatchZoo_latest/scripts/Letor/train_predict/train/concatenated/tested_configs_fold1

cd /projets/iris/PROJETS/WEIR/code/3rdYear/MatchZoo_latest/MatchZoo/

"""
cmd = "srun -n1 -N1 /projets/iris/PROJETS/WEIR/python_env/inherited/python3.5/bin/python3 matchzoo/main.py --phase train --model_file "
num = 0
new_script = True
limit = 12
for file in tqdm(os.listdir(output)):
    num += 1
    if new_script:
        out = open(os.path.join(scripts, "run_accnnm_"+str(num)+".sh"), 'w')
        out.write(header.replace("<num>", str(num)))
        new_script = False
    line = cmd + os.path.join(output, file).replace("/home/thiziri/Documents/DOCTORAT/OSIRIM", '') + \
           " > /projets/iris/PROJETS/WEIR/code/3rdYear/MatchZoo_latest/MatchZoo/examples/Letor/train_logs/concatenated/tested_configs_fold1/" + file.replace(".config", "_cpu.log") + '&'
    out.write(line + '\n')
    if num % 2 == 0:
        out.write('wait\n')
    if num % limit == 0 or len(os.listdir(output)) - num < limit:
        new_script = True
        # if len(os.listdir(output)) - num < limit:
out.close()

  0%|          | 0/2304 [00:00<?, ?it/s]

  1%|          | 14/2304 [00:00<00:17, 129.07it/s]

  2%|▏         | 38/2304 [00:00<00:12, 180.67it/s]

  3%|▎         | 62/2304 [00:00<00:11, 199.29it/s]

  4%|▍         | 97/2304 [00:00<00:09, 223.37it/s]

...

 96%|█████████▋| 2218/2304 [00:08<00:00, 267.12it/s]

 97%|█████████▋| 2245/2304 [00:08<00:00, 266.92it/s]

 99%|█████████▊| 2270/2304 [00:08<00:00, 265.92it/s]

100%|█████████▉| 2294/2304 [00:08<00:00, 265.44it/s]

100%|██████████| 2304/2304 [00:08<00:00, 259.32it/s]

In [75]:
batch = open("/home/thiziri/Documents/DOCTORAT/OSIRIM/projets/iris/PROJETS/WEIR/code/3rdYear/MatchZoo_latest/scripts/Letor/train_predict/train/concatenated/tested_configs_fold1/run_gpu_scripts.sh", 'w')
batch.write("""#!/bin/sh
#SBATCH --job-name=run_batches              # Nom du Job
#SBATCH --ntasks=1                       # Nombre de Tasks : 1
#SBATCH --cpus-per-task=4                # Allocation de 6 CPU par Task
#SBATCH --output=run_batches.out
#SBATCH --partition=24CPUNodes
#SBATCH --mem-per-cpu=7500M

#SBATCH --mail-type=ALL                  # Notification par mail des évènements concernant le job : début d’exécution, fin,…
#SBATCH --mail-user=thiziri.belkacem@irit.fr

# /projets/iris/PROJETS/WEIR/code/3rdYear/MatchZoo_latest/scripts/Letor/train_predict/train/concatenated/tested_configs_fold1
""")
for file in os.listdir("/home/thiziri/Documents/DOCTORAT/OSIRIM/projets/iris/PROJETS/WEIR/code/3rdYear/MatchZoo_latest/scripts/Letor/train_predict/train/concatenated/tested_configs_fold1/in_cpu"):
    batch.write("srun -n1 -N1 in_cpu/"+file+'\n')
batch.close()